In [19]:
!git clone https://github.com/Wukkkinz-0725/animalImage_classification.git

Cloning into 'animalImage_classification'...
remote: Enumerating objects: 18600, done.
remote: Counting objects: 100% (18600/18600), done.
remote: Compressing objects: 100% (49/49), done.
remote: Total 18600 (delta 18556), reused 18582 (delta 18549), pack-reused 0
Receiving objects: 100% (18600/18600), 13.78 MiB | 7.52 MiB/s, done.
Resolving deltas: 100% (18556/18556), done.
Updating files: 100% (18712/18712), done.


In [20]:
!pip install -q efficientnet_pytorch

  Preparing metadata (setup.py) ... done


In [2]:
import os
import random
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision

from torch.utils.data import Dataset, DataLoader, BatchSampler, random_split
from torchvision import transforms
from PIL import Image

In [3]:
os.chdir('./animalImage_classification/Released_Data')

## Baseline

In [11]:
# Create Dataset class for multilabel classification
class MultiClassImageDataset(Dataset):
    def __init__(self, ann_df, super_map_df, sub_map_df, img_dir, transform=None):
        self.ann_df = ann_df
        self.super_map_df = super_map_df
        self.sub_map_df = sub_map_df
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.ann_df)

    def __getitem__(self, idx):
        img_name = self.ann_df['image'][idx]
        img_path = os.path.join(self.img_dir, img_name)
        image = Image.open(img_path).convert('RGB')

        super_idx = self.ann_df['superclass_index'][idx]
        super_label = self.super_map_df['class'][super_idx]

        sub_idx = self.ann_df['subclass_index'][idx]
        sub_label = self.sub_map_df['class'][sub_idx]

        if self.transform:
            image = self.transform(image)

        return image, super_idx, super_label, sub_idx, sub_label

class MultiClassImageTestDataset(Dataset):
    def __init__(self, super_map_df, sub_map_df, img_dir, transform=None):
        self.super_map_df = super_map_df
        self.sub_map_df = sub_map_df
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self): # Count files in img_dir
        return len([fname for fname in os.listdir(self.img_dir)])

    def __getitem__(self, idx):
        img_name = str(idx) + '.jpg'
        img_path = os.path.join(self.img_dir, img_name)
        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        return image, img_name

In [52]:
train_ann_df = pd.read_csv('train_data.csv')
super_map_df = pd.read_csv('superclass_mapping.csv')
sub_map_df = pd.read_csv('subclass_mapping.csv')

train_img_dir = 'train_shuffle'
test_img_dir = 'test_shuffle'

image_preprocessing = transforms.Compose([
        transforms.RandomRotation(10),      # rotate +/- 10 degrees
        transforms.RandomHorizontalFlip(),  # reverse 50% of images
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
])

# Create train and val split
train_dataset = MultiClassImageDataset(train_ann_df, super_map_df, sub_map_df, train_img_dir, transform=image_preprocessing)
train_dataset, val_dataset = random_split(train_dataset, [0.9, 0.1])

# Create test dataset
test_dataset = MultiClassImageTestDataset(super_map_df, sub_map_df, test_img_dir, transform=image_preprocessing)

# Create dataloaders
batch_size = 64
train_loader = DataLoader(train_dataset,
                          batch_size=batch_size,
                          shuffle=True)

val_loader = DataLoader(val_dataset,
                        batch_size=batch_size,
                        shuffle=True)

test_loader = DataLoader(test_dataset,
                         batch_size=1,
                         shuffle=False)

In [49]:
def load_transform_single_image(image_path, transform):
    image = Image.open(image_path).convert('RGB')  # Ensure it's read in RGB format
    image = transform(image)
    return image


In [82]:
# Simple CNN
class CNN(nn.Module):
    def __init__(self):
        super().__init__()

        self.block1 = nn.Sequential(
                        nn.Conv2d(3, 32, 3, padding='same'),
                        nn.ReLU(),
                        nn.BatchNorm2d(32),
                        nn.Conv2d(32, 32, 3, padding='same'),
                        nn.ReLU(),
                        nn.BatchNorm2d(32),
                        nn.Conv2d(32, 32, 3, padding='same'),
                        nn.ReLU(),
                        nn.BatchNorm2d(32),
                        nn.MaxPool2d(2, 2)
                      )

        self.block2 = nn.Sequential(
                        nn.Conv2d(32, 64, 3, padding='same'),
                        nn.ReLU(),
                        nn.BatchNorm2d(64),
                        nn.Conv2d(64, 64, 3, padding='same'),
                        nn.ReLU(),
                        nn.BatchNorm2d(64),
                        nn.Conv2d(64, 64, 3, padding='same'),
                        nn.ReLU(),
                        nn.BatchNorm2d(64),
                        nn.MaxPool2d(2, 2)
                      )

        self.block3 = nn.Sequential(
                        nn.Conv2d(64, 128, 3, padding='same'),
                        nn.ReLU(),
                        nn.BatchNorm2d(128),
                        nn.Conv2d(128, 128, 3, padding='same'),
                        nn.ReLU(),
                        nn.BatchNorm2d(128),
                        nn.Conv2d(128, 128, 3, padding='same'),
                        nn.ReLU(),
                        nn.BatchNorm2d(128),
                        nn.MaxPool2d(2, 2)
                      )

        self.fc1 = nn.Linear(4*4*128, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3a = nn.Linear(128, 4)
        self.fc3b = nn.Linear(128, 88)

    def forward(self, x):
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = torch.flatten(x, 1) # flatten all dimensions except batch

        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        super_out = self.fc3a(x)
        sub_out = self.fc3b(x)
        return super_out, sub_out

class Trainer():
    def __init__(self, model, criterion, optimizer, train_loader, val_loader, test_loader=None, device='cuda', train_super=True, train_sub=True):
        self.model = model
        self.criterion = criterion
        self.optimizer = optimizer
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.test_loader = test_loader
        self.device = device
        self.train_super = train_super
        self.train_sub = train_sub

    def train_epoch(self):
        running_loss = 0.0
        for i, data in enumerate(self.train_loader):
            inputs, super_labels, sub_labels = data[0].to(self.device), data[1].to(self.device), data[3].to(self.device)

            self.optimizer.zero_grad()
            if self.train_super and self.train_sub:
                super_outputs, sub_outputs = self.model(inputs)
            elif self.train_super:
                super_outputs = self.model(inputs)
            else:
                sub_outputs = self.model(inputs)
            loss = 0
            if self.train_super:
                loss += self.criterion(super_outputs, super_labels)
            if self.train_sub:
                loss += self.criterion(sub_outputs, sub_labels)
            loss.backward()
            self.optimizer.step()

            running_loss += loss.item()

        print(f'Training loss: {running_loss / len(self.train_loader):.3f}')

    def validate_epoch(self):
        super_correct, sub_correct, total = 0, 0, 0
        running_loss = 0.0

        with torch.no_grad():
            for i, data in enumerate(self.val_loader):
                inputs, super_labels, sub_labels = data[0].to(self.device), data[1].to(self.device), data[3].to(self.device)

                if self.train_super and self.train_sub:
                    super_outputs, sub_outputs = self.model(inputs)
                elif self.train_super:
                    super_outputs = self.model(inputs)
                else:
                    sub_outputs = self.model(inputs)
                loss = 0
                if self.train_super:
                    loss += self.criterion(super_outputs, super_labels)
                    _, super_predicted = torch.max(super_outputs.data, 1)
                    super_correct += (super_predicted == super_labels).sum().item()
                if self.train_sub:
                    loss += self.criterion(sub_outputs, sub_labels)
                    _, sub_predicted = torch.max(sub_outputs.data, 1)
                    sub_correct += (sub_predicted == sub_labels).sum().item()
                total += super_labels.size(0)
                running_loss += loss.item()

        print(f'Validation Loss: {running_loss / len(self.val_loader):.3f}')
        if self.train_super:
            print(f'Validation Superclass Accuracy: {100 * super_correct / total:.2f} %')
        if self.train_sub:
            print(f'Validation Subclass Accuracy: {100 * sub_correct / total:.2f} %')

    def test(self, save_to_csv=False, return_predictions=False):
        self.model.eval()
        if not self.test_loader:
            raise NotImplementedError('test_loader not specified')

        superclass_predictions = {'image': [], 'superclass_index': []}
        subclass_predictions = {'image': [], 'subclass_index': []}

        with torch.no_grad():
            for i, data in enumerate(self.test_loader):
                inputs, img_name = data[0].to(self.device), data[1]

                if self.train_super and self.train_sub:
                    super_outputs, sub_outputs = self.model(inputs)
                elif self.train_super:
                    super_outputs = self.model(inputs)
                    sub_outputs = None
                elif self.train_sub:
                    super_outputs = None
                    sub_outputs = self.model(inputs)
                else:
                    continue  # Skip if neither superclass nor subclass is trained

                if self.train_super:
                    super_probs = F.softmax(super_outputs, dim=1)
                    superclass_predictions['superclass_probs'].append(super_probs.cpu().numpy())
                    superclass_predictions['image'].append(img_name[0])

                if self.train_sub:
                    sub_probs = F.softmax(sub_outputs, dim=1)
                    subclass_predictions['subclass_probs'].append(sub_probs.cpu().numpy())
                    subclass_predictions['image'].append(img_name[0])

        if save_to_csv:
            if self.train_super:
                superclass_df = pd.DataFrame(data=superclass_predictions)
                superclass_df.to_csv('superclass_prediction.csv', index=False)

            if self.train_sub:
                subclass_df = pd.DataFrame(data=subclass_predictions)
                subclass_df.to_csv('subclass_prediction.csv', index=False)

        if return_predictions:
            return superclass_predictions, subclass_predictions


In [77]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from efficientnet_pytorch import EfficientNet

class CustomEfficientNetB7(nn.Module):
    def __init__(self, num_super_classes=4, num_sub_classes=88):
        super(CustomEfficientNetB7, self).__init__()
        # 加载预训练的EfficientNet-B7模型
        self.base_model = EfficientNet.from_pretrained('efficientnet-b7')

        # 获取原始最后一层的输入特征数
        in_features = self.base_model._fc.in_features

        # 为超类和子类定义分类头
        self.super_class_classifier = nn.Linear(in_features, num_super_classes)
        self.sub_class_classifier = nn.Linear(in_features, num_sub_classes)

    def forward(self, x):
        # 从基础模型提取特征
        features = self.base_model.extract_features(x)

        # 全局平均池化
        pooled_features = F.adaptive_avg_pool2d(features, 1).squeeze(-1).squeeze(-1)

        # 分类到超类和子类
        super_class_output = self.super_class_classifier(pooled_features)
        sub_class_output = self.sub_class_classifier(pooled_features)

        return super_class_output, sub_class_output

# 使用BCE损失函数
def custom_loss(super_class_output, super_labels, sub_class_output, sub_labels):
    bce_loss = nn.BCEWithLogitsLoss()
    super_class_loss = bce_loss(super_class_output, F.one_hot(super_labels, num_classes=4).float())
    sub_class_loss = bce_loss(sub_class_output, F.one_hot(sub_labels, num_classes=88).float())
    return super_class_loss + sub_class_loss


In [79]:
# Init model and trainer
device = 'cuda'
model = CustomEfficientNetB7().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)
trainer = Trainer(model, criterion, optimizer, train_loader, val_loader, test_loader)

Loaded pretrained weights for efficientnet-b7


In [80]:
# Training loop
for epoch in range(10):
    print(f'Epoch {epoch+1}')
    trainer.train_epoch()
    trainer.validate_epoch()
    print('')

print('Finished Training')

Epoch 1
Training loss: 3.966
Validation Loss: 2.979
Validation Superclass Accuracy: 89.08 %
Validation Subclass Accuracy: 31.96 %

Epoch 2
Training loss: 2.425
Validation Loss: 2.335
Validation Superclass Accuracy: 92.41 %
Validation Subclass Accuracy: 45.25 %

Epoch 3
Training loss: 1.856
Validation Loss: 2.063
Validation Superclass Accuracy: 92.09 %
Validation Subclass Accuracy: 49.05 %

Epoch 4
Training loss: 1.542
Validation Loss: 1.841
Validation Superclass Accuracy: 93.35 %
Validation Subclass Accuracy: 55.70 %

Epoch 5
Training loss: 1.219
Validation Loss: 1.783
Validation Superclass Accuracy: 94.94 %
Validation Subclass Accuracy: 57.59 %

Epoch 6
Training loss: 1.083
Validation Loss: 1.725
Validation Superclass Accuracy: 93.04 %
Validation Subclass Accuracy: 58.86 %

Epoch 7
Training loss: 0.998
Validation Loss: 1.701
Validation Superclass Accuracy: 93.83 %
Validation Subclass Accuracy: 62.50 %

Epoch 8
Training loss: 0.842
Validation Loss: 1.578
Validation Superclass Accuracy:

In [89]:
trainer.model.eval()
if not trainer.test_loader:
    raise NotImplementedError('test_loader not specified')

superclass_predictions = {'image': [], 'superclass_index': [], 'probs': []}
subclass_predictions = {'image': [], 'subclass_index': [], 'probs': []}

with torch.no_grad():
    for i, data in enumerate(trainer.test_loader):
        inputs, img_name = data[0].to(trainer.device), data[1]

        if trainer.train_super and trainer.train_sub:
            super_outputs, sub_outputs = trainer.model(inputs)
        elif trainer.train_super:
            super_outputs = trainer.model(inputs)
            sub_outputs = None
        elif trainer.train_sub:
            super_outputs = None
            sub_outputs = trainer.model(inputs)
        else:
            continue  # Skip if neither superclass nor subclass is trained

        if trainer.train_super:
            super_probs = F.softmax(super_outputs, dim=1)
            superclass_predictions['probs'].append(super_probs.cpu().numpy())
            superclass_predictions['image'].append(img_name[0])

        if trainer.train_sub:
            sub_probs = F.softmax(sub_outputs, dim=1)
            subclass_predictions['probs'].append(sub_probs.cpu().numpy())
            subclass_predictions['image'].append(img_name[0])

if trainer.train_super:
    superclass_df = pd.DataFrame(data=superclass_predictions)
    superclass_df.to_csv('superclass_prediction_with_prob.csv', index=False)

if trainer.train_sub:
    subclass_df = pd.DataFrame(data=subclass_predictions)
    subclass_df.to_csv('subclass_prediction_with_prob.csv', index=False)

ValueError: ignored

In [95]:
del superclass_predictions(['superclass_index'])

SyntaxError: ignored

In [102]:
subclass_predictions.keys()

dict_keys(['image', 'subclass_index', 'probs'])

In [103]:
output_df = pd.DataFrame({'ID': subclass_predictions['image'], 'probs': subclass_predictions['probs']})

In [104]:
output_df.to_csv('sub_class_probs.csv', index=False)